# Gao's NDRS

1. Setup($1^k$)
2. KeyGen($\mathbb{P}$)
3. Sign($\mathbb{P}, R, {sk}_j, \mu$)
4. Verify($\mathbb{P}, R, \mu, \sigma$)
5. EvidenceGen($\mathbb{P}, R, {sk}_i, \mu, \sigma$)
6. EvidenceCheck($\mathbb{P}, R, i, \xi_i, \mu, \sigma$)

## Import

In [9]:
from random import *
from ndrs import *

## NDRS

In [2]:
def original(k: int, N: int, msg: str):
    # Create a new NDRS crypto object
    ndrs = NDRS(k)
    print(ndrs)

    keys = list[KeyPair]()
    for i in range(N):
        # Add a new signer
        keys.append(ndrs.key_gen())
        print(f"The key of Signer {i} added: {keys[i]}")

    print()

    # A real signer
    signer_index = randint(0, N-1)
    signer = keys[signer_index]
    
    # Choose the random amount of other signers except the real signer
    other_signer_indices = sample([i for i in range(N) if i != signer_index], randint(1, N-1))
    other_signers = [keys[i].pk for i in other_signer_indices]

    # Sign a message
    print(f"Signers {other_signer_indices} help the real signer {signer_index} to sign the message: {msg}")
    sig = ndrs.sign(signer, other_signers, msg)
    print(f"Signature: {sig}")
    
    # Verify the signature
    if ndrs.verify(msg, sig):
        print("Signature is valid (accept)")
    else:
        print("Signature is invalid (reject)")

    print()

    # Generate an evidence
    evi = ndrs.evidence_gen(signer, msg, sig)
    print(f"Evidence for real signer: {evi}")

    # Verify the evidence
    try:
        if ndrs.evidence_check(signer, msg, sig, evi):
            print("Evidence is valid (confirmation)")
        else:
            print("Evidence is invalid (disavowal)")
    except Exception as e:
        print(f"Evidence is invalid (failed)")

## Attack

In [18]:
def attack(k: int, N: int, msg: str):
    # Create a new NDRS crypto object
    ndrs = Frameable_NDRS(k)
    print(ndrs)

    keys = list[KeyPair]()
    for i in range(N):
        # Add a new signer
        keys.append(ndrs.key_gen())
        print(f"The key of Signer {i} added: {keys[i]}")

    print()

    # A real signer
    signer_index = randint(0, N-1)
    signer, fake_skey = ndrs.fake_skey_gen(keys[signer_index])
    print(f"The real secret key of the malicious signer: {keys[signer_index].sk}")
    print(f"The fake secret key of the malicious signer: {fake_skey}")
    print(f"And the public key of the malicious signer has been changed to: {keys[signer_index].pk}")
    
    # Choose the random amount of other signers except the real signer
    other_signer_indices = sample([i for i in range(N) if i != signer_index], randint(1, N-1))
    other_signers = [keys[i].pk for i in other_signer_indices]
    framed_idx = choice(other_signer_indices)

    print()

    # Sign a message
    print(f"Signers {other_signer_indices} help the malicious signer {signer_index} that want to frame the honest signer {framed_idx} to sign the message: {msg}")
    sig = ndrs.frameably_sign(KeyPair(signer.pk, fake_skey), other_signers, msg, other_signer_indices.index(framed_idx))
    print(f"Signature: {sig}")
    
    # Verify the signature
    if ndrs.verify(msg, sig):
        print("Signature is valid")
    else:
        print("Signature is invalid")

    print()

    # Generate an evidence for the malicious signer
    evi = ndrs.evidence_gen(signer, msg, sig)
    print(f"Evidence for malicious signer: {evi}")

    # Verify the evidence of the malicious signer
    try:
        if ndrs.evidence_check(signer, msg, sig, evi):
            print("Evidence is valid (confirmation)")
        else:
            print("Evidence is invalid (disavowal)")
    except Exception as e:
        print(f"Evidence is invalid (failed)")

    print()

    # Generate an evidence for the honest signer
    evi = ndrs.evidence_gen(keys[framed_idx], msg, sig)
    print(f"Evidence for honest signer: {evi}")

    # Verify the evidence of the honest signer
    try:
        if ndrs.evidence_check(keys[framed_idx], msg, sig, evi):
            print("Evidence is valid (confirmation)")
        else:
            print("Evidence is invalid (disavowal)")
    except Exception as e:
        print(f"Evidence is invalid (failed)")

## Security Proof

In [ ]:
original(10, 5, "我最愛密碼學了！")

In [ ]:
attack(10, 5, "我最愛密碼學了！")